# DUSP1 Confirmation and Visualization Notebook

This notebook processes experimental data from Big-FISH and CellProperties CSV files to classify and visualize DUSP1 smiFISH spots. The analysis is modular, using dedicated classes for loading data, performing signal-to-noise classification, measurement extraction, and filtering.

---

### **Input**
- Big-FISH CSV files (`spots`, `clusters`)
- CellProperties CSV files (`cell_props`, `cell_results`)

---

### **Workflow Overview**

1. **Load Experimental Data**
   - Use `DUSP1AnalysisManager` to identify and load datasets from HDF5 files or local CSVs.
   - Extract file paths from a log directory if not directly provided.

2. **Classify Spots by Signal Quality (SNR Analysis)**
   - Use `SNRAnalysis` to perform:
     - **Weighted SNR thresholding** using Big-FISH `'snr'` values with a percentile cutoff (e.g., 20th percentile within the range 2–5).
     - **Absolute thresholding** using a fixed cutoff on the Big-FISH `'snr'` value.
     - **MG SNR calculation**:  
       A more localized method that accounts for subcellular context. It computes:
       ```
       MG_SNR = (signal - mean) / std
       ```
       where `mean` and `std` are drawn from either the nuclear or cytoplasmic region of the same cell depending on the spot's location. This method reflects more accurate signal variation within individual cells.

   - Adds boolean flags and comparison columns (`MG_pass`, `Abs_pass`, `Weighted_pass`) to aid classification.

3. **Spot and Cell Measurement Extraction**
   - Use `DUSP1Measurement` to:
     - Quantify spot-level and cell-level metrics.
     - Support optional filtering using SNR results.
     - Append `unique_cell_id` for downstream aggregation and visualization.

4. **Filter and Save Processed Data**
   - Apply quality filters to retain only confident spots and cells.
   - Final outputs include:
     - `Finalspots`
     - `Finalclusters`
     - `Finalcellprops`
     - `SSITcellresults`

5. **Visualization**
   - Use `DUSP1DisplayManager` to inspect and validate each analysis step.
   - View raw data, spot overlays, SNR distributions, and per-cell metrics.

---

### **Core Classes**

### **Core Classes**

- **`DUSP1AnalysisManager`**  
  Loads HDF5 datasets or existing CSVs. Manages filepaths and FOV indexing. Exports intermediate and final data to CSV.

- **`SNRAnalysis`**  
  Performs three types of signal quality assessment:  
  1. **Weighted SNR**: percentile-based filtering on Big-FISH `snr`.  
  2. **Absolute SNR**: fixed-threshold filtering on Big-FISH `snr`.  
  3. **MG SNR**: cell-aware SNR using cytoplasmic or nuclear mean & std from CellProperties.  
  Adds boolean flags (`MG_pass`, `Abs_pass`, `Weighted_pass`) and diagnostic columns.

- **`DUSP1Measurement`**  
  Aggregates spot- and cluster-level data to the cell level. Calculates per-cell metrics (e.g. total spots, nuclear vs cytoplasmic counts) and attaches unique IDs for downstream grouping.

- **`DUSP1Filtering`**  
  Applies the various SNR‐based quality filters to spot and cluster data:  
  - Subsets spot tables to only those that pass both absolute and MG SNR checks.  
  - Prunes whole cells if too few spots remain or if cell‐level metrics fall outside expected ranges.  
  - Writes out the cleaned tables:  
    - `Finalspots.csv`  
    - `Finalclusters.csv`  
    - `Finalcellprops.csv`
    - `SSITcellresults.csv`  

- **`DUSP1DisplayManager`**  
  Interactive and static visual tools:  
  - **Segmentation overlays**: show nuclei/cytoplasm masks on raw images.  
  - **Spot detection review**: overlay spot foci on images.  
  - **Crop display**: view spot thumbnails for manual QC.

- **`PostProcessingPlotter`**  
  Statistical summary plots on the final, filtered data:  
  1. **TS‐Frequency Bar Chart**: fraction of cells with 1, 2, 3, or ≥4 transcription sites over time.  
  2. **Ridge (Joy) Plots**: stacked distributions for nuclear, cytoplasmic, and total mRNA counts, with control CDF thresholds annotated.  
  3. **Line Plots (Mean ± SD)**: nuclear and cytoplasmic mRNA counts over time, including the 0 min control baseline, with error bands indicating standard deviation.

---

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask.array as da
import os
import sys
import logging
import seaborn as sns
import datetime
from time import sleep

# Today's date
today = datetime.date.today()
# Format date as 'Jun03' (for example)
date_str = today.strftime("%b%d")

logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.Analysis_DUSP1 import DUSP1AnalysisManager, SNRAnalysis, DUSP1Measurement, DUSP1_filtering, DUSP1DisplayManager, PostProcessingPlotter

**`DUSP1AnalysisManager`** 
   - Manages HDF5 file access.
   - Extracts file paths from a log directory (if no direct locations are provided).
   - Provides methods to select an analysis (by name) and load datasets from HDF5 files.
   - Saves datasets as CSV.

In [ ]:
loc = None
log_location = r'/Volumes/share/Users/Eric/GR_DUSP1_reruns'
save_dir = r'/Volumes/share/Users/Eric/GR_DUSP1_AllData/DUSP1_FinalAnalysis_062425'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define filtering method and thresholds
method = 'mg_abs'            # options: 'mg', 'absolute', 'mg_abs', 'weighted', 'rf', 'none'
mac = True

In [ ]:
# am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
# am.list_analysis_names()

**`Combine Analyses function`**

In [ ]:
def make_combined_manager(loc, log_location, mac, analysis_names):
    """
    Build a single DUSP1AnalysisManager that “knows about” all of the N analyses.
    
    Parameters:
      - loc, log_location, mac: passed straight through to each temp manager
      - analysis_names: list of the analysis_name strings for each analysis
    Returns:
      - a DUSP1AnalysisManager whose .location and .analysis_names
        cover every file + analysis in analysis_names
    """
    all_paths = []
    all_anames = []

    for name in analysis_names:
        temp = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=mac)
        try:
            temp.select_analysis(name)
            # grab its now-filtered list of files + names
            all_paths   .extend(temp.location)
            all_anames  .extend(temp.analysis_names)
        finally:
            temp.close()    # <— guarantee no HDF5 leak
    # now build the combined
    combo = DUSP1AnalysisManager(location=all_paths, mac=mac)
    # manually inject the unioned analysis names
    combo.analysis_names = all_anames
    return combo

**`DUSP1 Replica D 3hr 100nM time-sweep R1 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_D_slide1_BFmean_061825',
    'DUSP1_D_slide2_BFmean_061825',
    'DUSP1_D_slide3_BFmean_061825',
    'DUSP1_D_slide4_BFmean_061825',
    'DUSP1_D_slide5_BFmean_061825',
    'DUSP1_D_slide6_BFmean_061825',
    'DUSP1_D_slide7_BFmean_061825',
    'DUSP1_D_slide8_BFmean_061825',
    'DUSP1_D_slide9_BFmean_061825',
    'DUSP1_D_slide10_BFmean_061825',
    'DUSP1_D_slide11_BFmean_061825',
    'DUSP1_D_slide12_BFmean_061825',        
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 10
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) main display ──────────────────────────────────────────────────────────
# # build a fresh manager so the display code can find every HDF5
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_D_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica E 3hr 100nM time-sweep R2 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_E_slide1_BFmean_061825',
    'DUSP1_E_slide2_BFmean_061825',
    'DUSP1_E_slide3_BFmean_061825',
    'DUSP1_E_slide4_BFmean_061825',
    'DUSP1_E_slide5_BFmean_061825',
    'DUSP1_E_slide6_BFmean_061825',
    'DUSP1_E_slide7_BFmean_061825',
    'DUSP1_E_slide8_BFmean_061825',
    'DUSP1_E_slide9_BFmean_061825',
    'DUSP1_E_slide10_BFmean_061825',
    'DUSP1_E_slide11_BFmean_061825',
    'DUSP1_E_slide12_BFmean_061825',        
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 20
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) main display ──────────────────────────────────────────────────────────
# # build a fresh manager so the display code can find every HDF5
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_E_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica F 3hr 100nM time-sweep R3 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_F_slide1_BFmean_061825',
    'DUSP1_F_slide2_BFmean_061825',
    'DUSP1_F_slide3_BFmean_061825',
    'DUSP1_F_slide4_BFmean_061825',
    'DUSP1_F_slide5_BFmean_061825',
    'DUSP1_F_slide6_BFmean_061825',
    'DUSP1_F_slide7_BFmean_061825',
    'DUSP1_F_slide8_BFmean_061825',
    'DUSP1_F_slide9_BFmean_061825',
    'DUSP1_F_slide10_BFmean_061825',
    'DUSP1_F_slide11_BFmean_061825',
    'DUSP1_F_slide12_BFmean_061825',       
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 30
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) main display ──────────────────────────────────────────────────────────
# # build a fresh manager so the display code can find every HDF5
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_F_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica M 3hr 100nM time-sweep Partial Replica - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_M_slide1_BFmean_061825',
    'DUSP1_M_slide2_BFmean_061825',
    'DUSP1_M_slide3_BFmean_061825',      
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 40
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) main display ──────────────────────────────────────────────────────────
# # build a fresh manager so the display code can find every HDF5
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_M_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica N 3hr 100nM time-sweep Partial Replica - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_N_slide1_BFmean_061825',
    'DUSP1_N_slide2_BFmean_061825',
    'DUSP1_N_slide3_BFmean_061825',
    'DUSP1_N_slide4_BFmean_061825',
    'DUSP1_N_slide5_BFmean_061825',
    'DUSP1_N_slide6_BFmean_061825',
    'DUSP1_N_slide7_BFmean_061825',      
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 50
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) main display ──────────────────────────────────────────────────────────
# # build a fresh manager so the display code can find every HDF5
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_N_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

# Time sweep for 100nM Dex
print("\n>>> Time sweep for 100nM Dex")
plotter.plot_time_sweep(
    dex_conc=100,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica J 3hr time-concentration sweep R1 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_J_slide1_BFmean_061825',
    'DUSP1_J_slide2_BFmean_061825',
    'DUSP1_J_slide3_BFmean_061825',
    'DUSP1_J_slide4_BFmean_061825',
    'DUSP1_J_slide5_BFmean_061825',
    'DUSP1_J_slide6_BFmean_061825',
    'DUSP1_J_slide7_BFmean_061825',
    'DUSP1_J_slide8_BFmean_061825',
    'DUSP1_J_slide9_BFmean_061825',
    'DUSP1_J_slide10_BFmean_061825',
    'DUSP1_J_slide11_BFmean_061825',
    'DUSP1_J_slide12_BFmean_061825',
    'DUSP1_J_slide13_BFmean_061825',
    'DUSP1_J_slide14_BFmean_061825',
    'DUSP1_J_slide15_BFmean_061825',
    'DUSP1_J_slide16_BFmean_061825',
    'DUSP1_J_slide17_BFmean_061825',
    'DUSP1_J_slide18_BFmean_061825',
    'DUSP1_J_slide19_BFmean_061825',        
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) apply your unique‐ID prefix logic ─────────────────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 60
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

# spots
max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

# clusters
max_cluster_id   = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix   = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) filtering ─────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)

filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) main display ──────────────────────────────────────────────────────────
# # build a fresh manager so the display code can find every HDF5
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post-filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_J_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

# Time + concentration sweeps for [0.3, 1, 10] nM Dex
print("\n>>> Time+Conc sweeps for [0.3, 1, 10] nM Dex")
plotter.plot_time_conc_sweep(
    conc_list=[0.3, 1, 10],
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica K 3hr time-concentration sweep R2 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_K_slide1_BFmean_061825',
    'DUSP1_K_slide2_BFmean_061825',
    'DUSP1_K_slide3_BFmean_061825',
    'DUSP1_K_slide4_BFmean_061825',
    'DUSP1_K_slide5_BFmean_061825',
    'DUSP1_K_slide6_BFmean_061825',
    'DUSP1_K_slide7_BFmean_061825',
    'DUSP1_K_slide8_BFmean_061825',
    'DUSP1_K_slide9_BFmean_061825',
    'DUSP1_K_slide10_BFmean_061825',
    'DUSP1_K_slide11_BFmean_061825',
    'DUSP1_K_slide12_BFmean_061825',
    'DUSP1_K_slide13_BFmean_061825',
    'DUSP1_K_slide14_BFmean_061825',
    'DUSP1_K_slide15_BFmean_061825',
    'DUSP1_K_slide16_BFmean_061825',
    'DUSP1_K_slide17_BFmean_061825',
    'DUSP1_K_slide18_BFmean_061825',
    'DUSP1_K_slide19_BFmean_061825',        
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 70
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) Main display ───────────────────────────────────────────────────────────
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_K_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

print("\n>>> Time+Conc sweeps for [0.3, 1, 10] nM Dex")
plotter.plot_time_conc_sweep(
    conc_list=[0.3, 1, 10],
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica L 3hr 100nM time-concentration sweep R2 - By Group`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_L_day1_90th_BF90th',
    'DUSP1_L_day2_mean_BFmean',
    'DUSP1_L_day3_25th_BF25th',
    'DUSP1_L_day4_25th_BF25th',       
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 80
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) Main display ───────────────────────────────────────────────────────────
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_L_BFmix'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

print("\n>>> Time+Conc sweeps for [0.3, 1, 10] nM Dex")
plotter.plot_time_conc_sweep(
    conc_list=[0.3, 1, 10],
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica G 75min concentration sweep R1 - By Group`**

In [ ]:
# 0) Define the three‐day analysis names for DUSP1_G
analysis_names_G = [
    'DUSP1_G_day1_BF75th',
    'DUSP1_G_day2_BF75th',
    'DUSP1_G_day3_75th',
]

# 1) Load & concatenate all three days in one pass
dm_G = make_combined_manager(loc, log_location, mac, analysis_names_G)
spots_df_G    = dm_G.select_datasets("spotresults",     dtype="dataframe")
clusters_df_G = dm_G.select_datasets("clusterresults",  dtype="dataframe")
props_df_G    = dm_G.select_datasets("cell_properties", dtype="dataframe")
dm_G.close()

print(f"Loaded {len(analysis_names_G)} days for G: "
      f"spots {spots_df_G.shape}, clusters {clusters_df_G.shape}, props {props_df_G.shape}")

# 2) SNRAnalysis → DUSP1Measurement
abs_threshold = 6
mg_threshold  = 3

snr_G = SNRAnalysis(spots_df_G, props_df_G, clusters_df_G,
                    abs_threshold=abs_threshold,
                    mg_threshold=mg_threshold)
merged_spots_G, merged_clusters_G, merged_cellprops_G = snr_G.get_results()

dusp_G = DUSP1Measurement(merged_spots_G, merged_clusters_G, merged_cellprops_G)
cell_level_results_G = dusp_G.measure(abs_threshold=abs_threshold,
                                      mg_threshold=mg_threshold)

# 3) Prefix unique IDs (prefix=90 for G replicate)
max_id_G   = merged_cellprops_G['unique_cell_id'].max()
digits_G   = len(str(max_id_G))
rep_prefix = 90
prefix_G   = rep_prefix * (10 ** digits_G)

for df in (merged_spots_G, merged_clusters_G, merged_cellprops_G, cell_level_results_G):
    df['unique_cell_id'] += prefix_G

# also prefix spot and cluster IDs
max_spot_id_G    = merged_spots_G['unique_spot_id'].max()
spot_prefix_G    = rep_prefix ** len(str(max_spot_id_G))
merged_spots_G   ['unique_spot_id'] += spot_prefix_G

max_cluster_id_G = merged_clusters_G['unique_cluster_id'].max()
cluster_prefix_G = rep_prefix ** len(str(max_cluster_id_G))
merged_clusters_G['unique_cluster_id'] += cluster_prefix_G

# 4) Filtering
filterer_G = DUSP1_filtering(method=method,
                             abs_threshold=abs_threshold,
                             mg_threshold=mg_threshold)
filtered_spots_G, filtered_clusters_G, filtered_cellprops_G, SSIT_G, removed_spots_G = \
    filterer_G.apply_all(
        spots=merged_spots_G,
        clusters=merged_clusters_G,
        cellprops=merged_cellprops_G
    )

# # 5) Image overlay display
# dm_disp_G = make_combined_manager(loc, log_location, mac, analysis_names_G)
# display_manager_G = DUSP1DisplayManager(
#     dm_disp_G,
#     cell_level_results=SSIT_G,
#     spots=filtered_spots_G,
#     clusters=filtered_clusters_G,
#     cellprops=filtered_cellprops_G,
#     removed_spots=removed_spots_G
# )
# display_manager_G.main_display()
# dm_disp_G.close()

# 6) Save only the final, post‐filter CSVs
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_G_BF75th'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSIT_G           .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots_G .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters_G.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops_G.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# 7) Post‐processing concentration sweep plot
#    auto-detect the single nonzero timepoint for this experiment
plotter_G = PostProcessingPlotter(
    clusters_df=filtered_clusters_G,
    cellprops_df=filtered_cellprops_G,
    ssit_df=SSIT_G,
    is_tpl=False,
)
times = sorted(SSIT_G['time'].unique())
nonzero = [t for t in times if t != 0]
if len(nonzero) != 1:
    raise ValueError(f"Expected exactly one non-zero time for G sweep, found {nonzero}")
timepoint_G = nonzero[0]

print(f"\n>>> Concentration sweep at t={timepoint_G} min for G replicate")
plotter_G.plot_conc_sweep(
    timepoint=timepoint_G,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica H 75min concentration sweep R2 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_H_slide1_BFmean_061825',
    'DUSP1_H_slide2_BFmean_061825',
    'DUSP1_H_slide3_BFmean_061825',
    'DUSP1_H_slide4_BFmean_061825',
    'DUSP1_H_slide5_BFmean_061825',
    'DUSP1_H_slide6_BFmean_061825',
    'DUSP1_H_slide7_BFmean_061825',
    'DUSP1_H_slide8_BFmean_061825',    
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    # free all HDF5 handles

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 110
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) Main display ───────────────────────────────────────────────────────────
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_H_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

times = sorted(SSITcellresults['time'].unique())
nonzero = [t for t in times if t != 0]
if len(nonzero) != 1:
    raise ValueError(f"Expected exactly one non-zero time for H sweep, found {nonzero}")
timepoint = nonzero[0]

print(f"\n>>> Concentration sweep at t={timepoint} min for H replicate")
plotter.plot_conc_sweep(
    timepoint=timepoint,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica I 75min concentration sweep R3 - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_I_slide1_BFmean_061825',
    'DUSP1_I_slide2_BFmean_061825',
    'DUSP1_I_slide3_BFmean_061825',
    'DUSP1_I_slide4_BFmean_061825',
    'DUSP1_I_slide5_BFmean_061825',
    'DUSP1_I_slide6_BFmean_061825',
    'DUSP1_I_slide7_BFmean_061825',
    'DUSP1_I_slide8_BFmean_061825',
    'DUSP1_I_slide9_BFmean_061825',      
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 120
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) Main display ───────────────────────────────────────────────────────────
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_I_BFmean'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) post-processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl= False,
)

times = sorted(SSITcellresults['time'].unique())
nonzero = [t for t in times if t != 0]
if len(nonzero) != 1:
    raise ValueError(f"Expected exactly one non-zero time for I sweep, found {nonzero}")
timepoint = nonzero[0]

print(f"\n>>> Concentration sweep at t={timepoint} min for I replicate")
plotter.plot_conc_sweep(
    timepoint=timepoint,
    save_dir=output_dir,
    display=True
)

**`DUSP1 Replica O Triptolide Time-sweep - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_O_BFmean_slide1_062025',
    'DUSP1_O_BFmean_slide2_062025',
    'DUSP1_O_BFmean_slide3_062025',
    'DUSP1_O_BFmean_slide4_062025',
    'DUSP1_O_BFmean_slide5_062025',
    'DUSP1_O_BFmean_slide6_062025',
    'DUSP1_O_BFmean_slide7_062025',
    'DUSP1_O_BFmean_slide8_062025',
    'DUSP1_O_BFmean_slide9_062025',      
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"spots column keys: {spots_df.keys()}")
print(f"  clusters:{clusters_df.shape}")
print(f"clusters column keys: {clusters_df.keys()}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df,
                        is_tpl=True)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 130
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold,
                           is_tpl=True)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# ─── 5) Main display ───────────────────────────────────────────────────────────
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_O_BFmean_tpl'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)
print("SSITcellresults keys:", SSITcellresults.keys())
print("Saved final results to:", output_dir)

# ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl=True
)

print(f"\n>>> 100nM Dex TPL for O replicate")
plotter.plot_time_sweep(
    dex_conc=100,
    # save_dir=output_dir,
    display=True
)

**`DUSP1 Replica P Triptolide Time-sweep - By Slide`**

In [ ]:
# ─── 1) pull in all slides  ──────────────────────────────────────────
analysis_names = [
    'DUSP1_P_BFmean_slide1_062025',
    'DUSP1_P_BFmean_slide2_062025',
    'DUSP1_P_BFmean_slide3_062025',
    'DUSP1_P_BFmean_slide4_062025',
    'DUSP1_P_BFmean_slide5_062025',
    'DUSP1_P_BFmean_slide6_062025',
    'DUSP1_P_BFmean_slide7_062025',
    'DUSP1_P_BFmean_slide8_062025',
    'DUSP1_P_BFmean_slide9_062025',
    'DUSP1_P_BFmean_slide10_062025',
    'DUSP1_P_BFmean_slide11_062025',
    'DUSP1_P_BFmean_slide12_062025',
    'DUSP1_P_BFmean_slide23_062025',
    'DUSP1_P_BFmean_slide14_062025',
    'DUSP1_P_BFmean_slide15_062025',
    'DUSP1_P_BFmean_slide16_062025',         
]

# build and use the combined manager
dm = make_combined_manager(loc, log_location, mac, analysis_names)

spots_df    = dm.select_datasets("spotresults",      dtype="dataframe")
clusters_df = dm.select_datasets("clusterresults",   dtype="dataframe")
props_df    = dm.select_datasets("cell_properties",  dtype="dataframe")
dm.close()    

print(f"Loaded and concatenated {len(analysis_names)} days:")
print(f"  spots:   {spots_df.shape}")
print(f"  clusters:{clusters_df.shape}")
print(f"  props:   {props_df.shape}")

# ─── 2) SNRAnalysis → DUSP1Measurement ────────────────────────────────────────
abs_threshold = 6
mg_threshold  = 3

snr = SNRAnalysis(spots_df, props_df, clusters_df,
                  abs_threshold=abs_threshold,
                  mg_threshold=mg_threshold)
merged_spots_df, merged_clusters_df, merged_cellprops_df = snr.get_results()

dusp = DUSP1Measurement(merged_spots_df,
                        merged_clusters_df,
                        merged_cellprops_df,
                        is_tpl=True)
cell_level_results = dusp.measure(abs_threshold=abs_threshold,
                                  mg_threshold=mg_threshold)

# ─── 3) Prefix unique IDs (prefix=70 for K replicate) ─────────────────────────
max_id     = merged_cellprops_df['unique_cell_id'].max()
num_digits = len(str(max_id))
rep_prefix = 140
prefix     = rep_prefix * (10 ** num_digits)

for df in (merged_spots_df, merged_clusters_df, merged_cellprops_df, cell_level_results):
    df['unique_cell_id'] += prefix

max_spot_id  = merged_spots_df['unique_spot_id'].max()
spot_prefix  = rep_prefix ** len(str(max_spot_id))
merged_spots_df   ['unique_spot_id'] += spot_prefix

max_cluster_id = merged_clusters_df['unique_cluster_id'].max()
cluster_prefix = rep_prefix ** len(str(max_cluster_id))
merged_clusters_df['unique_cluster_id'] += cluster_prefix

# ─── 4) Filtering ───────────────────────────────────────────────────────────────
filterer = DUSP1_filtering(method=method,
                           abs_threshold=abs_threshold,
                           mg_threshold=mg_threshold,
                           is_tpl=True)
filtered_spots, filtered_clusters, filtered_cellprops, SSITcellresults, removed_spots = \
    filterer.apply_all(spots=merged_spots_df,
                       clusters=merged_clusters_df,
                       cellprops=merged_cellprops_df)

# # ─── 5) Main display ───────────────────────────────────────────────────────────
# dm_disp = make_combined_manager(loc, log_location, mac, analysis_names)
# display_manager = DUSP1DisplayManager(
#     dm_disp,
#     cell_level_results=SSITcellresults,
#     spots=filtered_spots,
#     clusters=filtered_clusters,
#     cellprops=filtered_cellprops,
#     removed_spots=removed_spots
# )
# display_manager.main_display()
# dm_disp.close()

# ─── 6) Save final, post‐filter results to CSV ─────────────────────────────────
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
rep_string = 'DUSP1_P_BFmean_tpl'
base = f"{rep_string}_MG{mg_threshold}_Abs{abs_threshold}_{date_str}_{method}"

SSITcellresults  .to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
filtered_spots   .to_csv(os.path.join(output_dir, f"{base}_FinalSpots.csv"),      index=False)
filtered_clusters.to_csv(os.path.join(output_dir, f"{base}_FinalClusters.csv"),  index=False)
filtered_cellprops.to_csv(os.path.join(output_dir, f"{base}_FinalCellProps.csv"), index=False)

print("Saved final results to:", output_dir)

# ─── 7) Post‐processing plots with PostProcessingPlotter ──────────────────────
plotter = PostProcessingPlotter(
    clusters_df=filtered_clusters,
    cellprops_df=filtered_cellprops,
    ssit_df=SSITcellresults,
    is_tpl=True
)

print(f"\n>>> 100nM Dex TPL for P replicate")
plotter.plot_time_sweep(
    dex_conc=100,
    # save_dir=output_dir,
    display=True
)

## GR Confirmation

In [ ]:
from src.Analysis_GR import AnalysisManager, GR_Confirmation
loc = None
log_location = r'/Volumes/share/Users/Jack/All_Analysis'

# Measure/filter functions to create final GR dataframe for SSIT

def touching_border(df, image_height, image_width):
    """
    Returns a boolean Series indicating which cells are
    touching any image border (no margin).

    Expects df to have columns:
      'cell_bbox-0' => min_row
      'cell_bbox-1' => min_col
      'cell_bbox-2' => max_row (exclusive)
      'cell_bbox-3' => max_col (exclusive).
    """
    min_row = df['cell_bbox-0']
    min_col = df['cell_bbox-1']
    max_row = df['cell_bbox-2']
    max_col = df['cell_bbox-3']

    return (
        (min_row == 0) 
        | (min_col == 0) 
        | (max_row == image_height) 
        | (max_col == image_width)
    )

def filter_GR(name, rep_prefix: int):
    # 1) set up
    am = AnalysisManager(location=loc, log_location=log_location, mac=True)
    am.select_analysis(name)
    GR = GR_Confirmation(am)
    GR.get_data()

    # 2) get image dims for border‐touching
    image0 = GR.images[0]
    h, w = image0.shape[-2:]

    # 3) make a simple local index for each cell
    local_ids = np.arange(len(GR.cellprops))

    # 4) compute how many digits that index has
    max_local = local_ids.max()
    num_digits = len(str(max_local))

    # 5) build the rep‐specific offset
    offset = rep_prefix * (10 ** num_digits)

    # 6) assign your global unique_cell_id
    GR.cellprops['unique_cell_id'] = local_ids + offset

    # 7) remove border‐touching cells as before
    GR.cellprops['touching_border'] = touching_border(
        GR.cellprops, image_height=h, image_width=w
    )
    GR.cellprops = GR.cellprops[~GR.cellprops['touching_border']]

    am.close()

    # 8) return your SSIT‐style frame
    return measure_GR(GR.cellprops)

# Measure function for GR pre-SSIT dataframe 
def measure_GR(cellprops) -> pd.DataFrame:
    results = pd.DataFrame(columns=['cell_id', 'nuc_area', 'nucGRint', 'cytoGRint', 'time', 'dex_conc', 'replica'])
    
    # Sort cellprops by unique_cell_id
    props = cellprops.sort_values(by='unique_cell_id')
    # unique cell id
    cell_ids = props['unique_cell_id']
    # nuc area
    nuc_area = props['nuc_area']
    # avg int nuc
    nucGRint = props['nuc_intensity_mean-0']
    # avg int pseudocyto mask
    cytoGRint = props['cyto_intensity_mean-0']
    # time (experiment)
    time = props['time'] 
    # Dex conc
    dex_conc = props['Dex_Conc']
    # Replica
    replica = props['replica']
    results['cell_id'] = cell_ids
    results['nuc_area'] = nuc_area.values
    results['nucGRint'] = nucGRint.values
    results['cytoGRint'] = cytoGRint.values
    results['time'] = time.values
    results['dex_conc'] = dex_conc.values
    results['replica'] = replica.values

    return results    

In [ ]:
am = DUSP1AnalysisManager(location=loc, log_location=log_location, mac=True) 
am.list_analysis_names()

In [ ]:
GR_RepA = filter_GR('GR_IC_A_ER020725', rep_prefix=150)

In [ ]:
GR_RepB = filter_GR('GR_IC_B_ReRun021025', rep_prefix=160)

In [ ]:
GR_RepC = filter_GR('GR_IC_C_ER020725', rep_prefix=170)

In [ ]:
# Concatonate all DUSP1 Dex experiments & reset 'unique_cell_id'
GR_ALL = pd.concat(
[GR_RepA, GR_RepB, GR_RepC], ignore_index=True) # DUSP1_RepN, DUSP1_RepJ
GR_ALL['unique_cell_id'] = np.arange(len(GR_ALL))
# Save the final GR dataframe
output_dir = save_dir
os.makedirs(output_dir, exist_ok=True)
base = f"GR_ALL_{date_str}_{method}"
GR_ALL.to_csv(os.path.join(output_dir, f"{base}_SSITcellresults.csv"), index=False)
print("Saved final GR results to:", output_dir)